# HEP event graph visualiser
This visualiser represents the final state particles generated
from a Pythia shower of `p p > W' > W Z` as a graph.
The `W` decays hadronically, and the `Z` entirely into neutrinos.

The nodes are particles, whose size are scaled by their relative $p_T$ values.
Edges are formed between the particles with closest $\Delta R$. Thicker
edges represent closer $\Delta R$.

Red nodes are "signal" particles, which are descendants of the `W`, black
particles are background.

Zooming in, you can see the names of the particles labelling the nodes.
If you click the nodes, you can drag them around to get a sense of
how tightly connected they are with their neighbourhood.
Hovering over a node once clicked will display the particle's general properties.

There are a few limited settings you can fiddle with below. The higher the number
of edges you set, the longer it will take to form the graph.
The graph also may become quite crowded and hard to see, so there is an option
to increase the sparsity, although it may be harder to zoom in on and get ahold of
individual particles.

100 events have been uploaded.

**N.B.** There are no axes to these graphs,
the arrangement in space is just the result of a physics solver trying
to arrange the nodes sensibly based on their connections.
It treats the edges like springs and evolves the system into equilibrium.

## Settings for visitors to change and play with
Adjust the variables within this cell (and execute it) to change the generated graphs

In [1]:
KNN_VALUE = 15
EVENT_NUMBER = -1 # -1 is random
SPARSE = False

## Sets up my package for execution

In [7]:
!pip install ../

Processing /home/jovyan
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for cluster-gnn: filename=cluster_gnn-0.1.0-py2.py3-none-any.whl size=15725 sha256=db05dda3088be18e30965f8d2b8cbdc951fc1292e0d7f2bffd4d28dfc27b090c
  Stored in directory: /tmp/pip-ephem-wheel-cache-3rrhr_rp/wheels/fc/c4/49/78b5bd16ca276f2916d0829d47c131046b6e4575f7dd51e987
Successfully built cluster-gnn
  Attempting uninstall: cluster-gnn
    Found existing installation: cluster-gnn 0.1.0
    Uninstalling cluster-gnn-0.1.0:
      Successfully uninstalled cluster-gnn-0.1.0


## Visualiser code
Probably best not change the following code.

In [2]:
import os

import numpy as np
import scipy.sparse as sps
from pyvis.network import Network
from particle import Particle
import vector

from cluster_gnn import ROOT_DIR
from cluster_gnn.data import internal as Data
from cluster_gnn.features import build_features as Features

### Execute me repeatedly to render graphs over and over

In [3]:
with Data.EventLoader(ROOT_DIR + '/data/processed/events.hdf5', 'wboson') as evts:
    # set data object state to point at first event
    num_evts = len(evts)
    evt_num = EVENT_NUMBER if EVENT_NUMBER != -1 else np.random.randint(num_evts)
    evts.set_evt(evt_num)
    # form weighted edges
    pmu = evts.get_pmu()
    signal = evts.get_signal()
    sps_adj = sps.coo_matrix(
        Features.knn_adj(
            Features.deltaR_aff(pmu),
            k=KNN_VALUE,
            weighted=True,
            dtype=np.float32
            )
        )
    edge_idxs = zip(map(int, sps_adj.row),
                    map(int, sps_adj.col),
                    map(lambda wt: (float(wt) + 0.1)**-1, sps_adj.data)
                    )
    # get data from particle ids
    pcls = [Particle.from_pdgid(pdg) for pdg in evts.get_pdg()]
    labels = list(map(lambda pcl: pcl.name, pcls))
    titles = zip(
        map(lambda pcl: f'mass: {pcl.mass} MeV', pcls),
        map(lambda pcl: f'width: {pcl.width} MeV', pcls),
        map(lambda pcl: f'lifetime: {pcl.width} ns', pcls),
        map(lambda pcl: f'charge: {pcl.charge}', pcls),
        map(lambda pcl: f'isospin: {pcl.I}', pcls),
        map(lambda pcl: f'charge parity: {pcl.C}', pcls),
        map(lambda pcl: f'space parity: {pcl.P}', pcls),
        map(lambda pcl: f'total angular momentum: {pcl.J}', pcls),
    )
    titles = map(lambda title: '<br>'.join(title), titles)
    ids = list(range(len(pcls)))
    node_vals = Features._array_to_vec(pmu).pt
    node_vals = node_vals * 10.0 / node_vals.max()
    node_vals = tuple(float(val) for val in node_vals)
    # identify which are W boson
    groups = list(signal.astype(np.uint8))
    print(evts.get_evt_name())
net = Network(height=600, width=800, notebook=True, directed=False)
net.toggle_hide_edges_on_drag(False)
if SPARSE == True:
    net.barnes_hut()
else:
    net.barnes_hut(
        gravity=-10000,
        central_gravity=0.01,
        spring_length=200,
        spring_strength=0.015,
        overlap=1,
        damping=0.5,
    )
net.add_nodes(ids,
          label=labels,
          value=node_vals,
          color=['#162347' if group == 0 else '#dd4b39' for group in groups],
          title=list(titles),
          )
net.add_edges(edge_idxs)
net.show('ex.html')

event_092548
